<a href="https://colab.research.google.com/github/HungYangChang/ECSE551/blob/master/Miniproject1_combo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import relevant modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time


# Read Data Sets (Bankrupcy and Hepatitis)

In [ ]:
# Load bankruptcy data
url = "https://raw.githubusercontent.com/jonarsenault/ecse551data/master/bankrupcy.csv"
bank_data = pd.read_csv(url)

# Display some of the data
print(bank_data.head())

# Print size of data
bank_data.shape

In [ ]:
# Load hepatitis data
url = "https://raw.githubusercontent.com/jonarsenault/ecse551data/master/hepatitis.csv"
hep_data = pd.read_csv(url)

# Display some of the data
print(hep_data.head())

# Print size of data
hep_data.shape

# CHANGE LOG

List of changes made to code: \\
- Standardization was being performed on both the numerical and categorical data for the hepatitis data. I changed it to only the numerical data. 
- Split data function now works with numpy array as opposed to pandas dataframe.
- Now reporting training and testing accuracy for basic fit and test
- We only really care about the time it takes to fit the model, so I set the timers to measure that.
- Changed the standardization in the k-fold code to allow standardization of certain columns
Jon - 10/6
- Fixed the CE loss function. Problem was related to passing 0 to the log function. Even if it's technically defined for numbers very close to 0, the computer can't hold numbers that small.
- Added a section to try and

List of changes by James 10/4
- Not sure why you make split data function works with numpy array as opposed to pandas dataframe. But I cominbe taking random set and taking constant set to one function. TODO: make random works with numpy array
- Changed shuffle in the k-fold code to allow shuflling of certain row (parameter shuffle = True). #Good, but x_train and y_train need to be suffled at the same time! So I combined them, shuffled, then split
- Copy oringial data to data_kfold before shuffling in k-fold CV to prevent disturbing orinigal data
- #Question: why -1?  X_hep_cat_change[:, index_cat_columns] = X_hep_cat_change[:, index_cat_columns] - 1  (you can search Question in code) #Answer: I was just trying something! Typically binary variables are (0,1) but here we have (1,2) I was wondering if that made a difference (and it's still an open question)

List of changes by Anne 10/5
- Deleted irrelevant blocks
- Added CE function in Define functions block. But I don't think it works yet!! give me lots of NANs... Anyone help me out?? Thanks!! :p (Fixed: just passing very small values to the log function)
- Added CE in k-fold block, if the functions works, this should allow us to print and plot CE of each fold (Done!)
- Fixed bankruptcy basic fit! There is a small error in standardization section in k-fold function. It should not include the first column (bias one), so I fixed it, and it works okay now I think..
- Added option to NOT standardize in k-fold
- Added section to compare effect of standardization. Do you guys think you can work with this as a template to run other tests? There's separate block for each dataset. In each block, it copies the pre-processed data into two groups, we can modify each group with whatever we want to compare. Then run the k-fold loop (or just k-fold?) for each group to compare results.
- In order for the k-fold loop to be consice, I wrote an option to print result of each fold or not. (Good)
- Question: why is standardization NOT giving better results??? 
- I'll keep work on it tomorrow! To do: Fix CE, compare initial weight and stopping criteria k-fold, write results in report

List of changes by James 10/6 \
Note
- Return iteration time at Define funtion in fit. Now showing (1) average accuracy, (2) each accuracy, (3) iteration time after finsihing k-fold CV.
- Fix Basic Fit and test for bankruptcy

TODO \


# Define functions for linear classifier logistic regression

In [ ]:
# JA NOTE: I would combine these into one function, somehow. We can discuss.
# James note: combined split data to one funciton

def splitdata(data, perc_training, random_flag=True):
  """Split data into training and testing set using a random division or by taking a constant set"""
  if (random_flag == True):
    train_data = data.sample(frac = perc_training, random_state=np.random.RandomState())
    test_data = data.drop(train_data.index)
    # train_data = np.random.choice([False, True], len(data), replace=False ,p=[perc_training, (1-perc_training)])
  else:
    num_rows = data.shape[0]
    num_rows_train = int(num_rows * perc_training )
    num_rows_test = num_rows - num_rows_train
    train_data = data[:num_rows_train, :]
    test_data = data[:num_rows_test, :]
  return train_data, test_data

def standardization(data, training_data):
  """Standardize each column of input data"""
  data_standardized = (data - training_data.mean(axis=0))/training_data.std(axis=0)
  return data_standardized

def sigmoid(x):
  """Apply logistic activation function to input"""
  # Logistic activation function
  return 1 / (1 + np.exp(-x))

# Define fit, predict and accu_eval functions
class LogisticClassifier():
  """Class defining a logistic classifier"""
  #TODO read initial weights (will be discussed)
  def __init__(self, weights):
    """Constructor"""
    self._w = weights
  
  def fit(self, x_train, y_train, iter_time, epsilon = 1e-6):
    """Fit a logistic regression model to data"""

    # Lists to store weights and gradient at each iteration
    Weight_store = []

    # JA NOTE: Why do we not loop until the stopping criteria is reached as; Anne Note: revised oct.5
    # opposed to having a break statement later on
    iteration = 1
    delta_weights = 1e6

    while (delta_weights > epsilon) & (iteration < iter_time):
      #TODO selection of learning rate will be discussed

      # Set learning rate for this iteration
      learning_rate = 1/(1 + iteration)

      # Store current weights before updating
      weights_previous = self._w

      # Compute gradient of cross-entropy loss
      gradient = np.sum(
      x_train * (y_train - sigmoid(np.dot(x_train, weights_previous))), axis=0
      ).reshape(-1, 1)

      # Update weights
      self._w = weights_previous + learning_rate * gradient

      # Compute change in weights
      delta_weights = np.linalg.norm(self._w - weights_previous) ** 2    
      # TODO stopping criteria (choise of epsilon will be discussed)
      # TODO Could also try stopping when loss stops decreasing

      # Store weight and CE
      Weight_store.append(self._w.flatten()) 

      iteration += 1  

    return Weight_store, iteration

    if ((iteration+1)==iter_time):
      print ("Job Done, but failed to converge at ", iter_time, "times iteration")
    else:
      print ("Model converges at", iteration+1 ,"times iteration")

  def predict(self, x_test):
    """Predict the class labels of a given set of samples"""    
    y_pred_prob = sigmoid(np.dot(x_test, self._w))
    y_pred = np.where(y_pred_prob < 0.5, 0, 1)
    return y_pred
  

  def accu_eval(self, y_test, y_pred):
    """Compute accuracy of model""" 
    accuracy = np.count_nonzero(y_test == y_pred) / len(y_test)
    return accuracy

  def cross_entropy_loss(self, x_test, y_test):
    """Compute cross entropy """

    y_pred_prob = sigmoid(np.dot(x_test,self._w))
    y_pred_prob_m1 = 1 - y_pred_prob

    # Replace small values in both with 1e-5 to avoid NAN (log0)
    y_pred_prob = np.where(y_pred_prob < 1e-5, 1e-5, y_pred_prob)
    y_pred_prob_m1 = np.where(y_pred_prob_m1 < 1e-5, 1e-5, y_pred_prob_m1)

    loss_0 = y_test * np.log(y_pred_prob)
    loss_1 = (1-y_test) * np.log(y_pred_prob_m1)

    loss = -np.sum(loss_0 + loss_1)
    return loss


# K-fold Cross Validation

In [ ]:
# JA NOTE : I think this function should output a list of the accuracy and CE loss for every fold

def k_fold_Cross(test, x_train, y_train, k=10, standardize=True, index_to_standardize=None, shuffle=True, printresult=True):
  """ k-fold cross validation to estimate performance with accuracy"""

  # Shuffle data to introcude randomness to trials
  if (shuffle==True):
    full_array = np.concatenate((x_train, y_train), axis=1)
    np.random.shuffle(full_array)
    x_train = full_array[:,:-1]
    y_train = full_array[:,[-1]]

  #int() is floor funcion. If it's not aliquot, just insert all of remaining data to last fold
  fold_size = int(len(x_train)/k) 
  if (printresult==True):
    print ("Now doing k-fold cross validation, fold size= {}, x train shape = {}, y train shape = {}".format(fold_size, x_train.shape, y_train.shape))

  fold_accuracy_store = []
  Weight_store_fold = []
  fold_CE_store = []
  iteration_store_fold= []

  for fold_number in range(k):
    index_start = fold_size*fold_number
    index_end = fold_size*(fold_number+1)

    # for last fold (fold number k)
    if (fold_number == (k-1)):
      x_train_fold = x_train[:index_start,:]
      y_train_fold = y_train[:index_start,:]
      x_validation_fold = x_train[index_start:,:]
      y_validation_fold = y_train[index_start:,:]
    # for non-last fold
    else:
      x_train_fold = np.concatenate((x_train[:index_start,:], x_train[index_end:,:]),axis=0)
      y_train_fold = np.concatenate((y_train[:index_start,:], y_train[index_end:,:]),axis=0)
      x_validation_fold = x_train[index_start:index_end,:]
      y_validation_fold = y_train[index_start:index_end,:]

    # NOT standardize the data
    if ~standardize:
      x_train_fold_standardized = x_train_fold.copy()
      x_validation_fold_standardized = x_validation_fold.copy()

    # Standardize the data
    if (index_to_standardize is not None) and standardize:
      x_train_fold_standardized = x_train_fold.copy()
      x_train_fold_standardized[:,index_to_standardize] = standardization(
          x_train_fold[:,index_to_standardize], 
          x_train_fold[:,index_to_standardize]
      )
      x_validation_fold_standardized = x_validation_fold.copy()
      x_validation_fold_standardized[:,index_to_standardize] = standardization(
          x_validation_fold[:,index_to_standardize], 
          x_train_fold[:,index_to_standardize]
      )
      #print('section 1 ran')
    elif standardize==True:
      x_train_fold_standardized = x_train_fold.copy()
      x_train_fold_standardized[:,1:] = standardization(x_train_fold[:,1:], x_train_fold[:,1:])
      x_validation_fold_standardized = x_validation_fold.copy()
      x_validation_fold_standardized[:,1:] = standardization(x_validation_fold[:,1:], x_train_fold[:,1:])
      #print('section 2 ran')

    # Fit the model
    t_start = time.time()
    weights, iteration = test.fit(x_train_fold_standardized, y_train_fold, iter_time = 10000)
    t_end = time.time()

    Weight_store_fold.append(weights)
    iteration_store_fold.append(iteration)
    y_pred_train = test.predict(x_train_fold_standardized)
    y_pred_test = test.predict(x_validation_fold_standardized)
    accuracy_train = test.accu_eval(y_train_fold, y_pred_train)
    accuracy_test = test.accu_eval(y_validation_fold, y_pred_test)
    fold_accuracy_store.append(accuracy_test)
    
    cross_E=test.cross_entropy_loss(x_validation_fold_standardized,y_validation_fold)

    fold_CE_store.append(cross_E)
     
    if (printresult==True):
      print(f"### Fold number {fold_number+1} ###")
      print(f"Execution time: {t_end-t_start:.3f}s")
      print(f"Training Accuracy: {100*accuracy_train:.2f} %")
      print(f'Testing Accuracy: {100*accuracy_test:.2f} %')
      print(f'Cross-entropy loss CE: {cross_E}')

  cross_accuracy = sum(fold_accuracy_store)/k
  if (printresult==True):
    print (f"Mean testing accuracy is {cross_accuracy*100:.2f} %" )
  
  return fold_accuracy_store, Weight_store_fold, fold_CE_store, iteration_store_fold


# Basic fit and test of Hepatitis data

In [ ]:
# Indices of features
index_cat_columns = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18])
index_numerical_columns = np.array([0, 13, 14, 15, 16, 17])

# Create original set of features
X_hep_original_no_bias = hep_data.iloc[:, :-1].to_numpy()
X_hep_original = np.insert(
    X_hep_original_no_bias, 0, np.ones(X_hep_original_no_bias.shape[0]), axis=1
)
y_hep_original = hep_data.iloc[:, -1].to_numpy().reshape(-1, 1)

# Account for extra column for bias
index_numerical_columns = index_numerical_columns + 1
index_cat_columns = index_cat_columns + 1

# Create an instance of the model object
initial_weights = np.zeros((X_hep_original.shape[1], 1))
model = LogisticClassifier(initial_weights)

# Split the data
X_train, X_test = splitdata(X_hep_original, 0.8, False)
y_train, y_test = splitdata(y_hep_original, 0.8, False)

# Standardize the data
X_train_standardize = X_train.copy()
X_test_standardize = X_test.copy()

X_train_standardize[:, index_numerical_columns] = standardization(
    X_train[:, index_numerical_columns], X_train[:, index_numerical_columns]
)
X_test_standardize[:, index_numerical_columns] = standardization(
    X_test[:, index_numerical_columns], X_train[:, index_numerical_columns]
)

# X_train_standardize[:, index_cat_columns] = X_train_standardize[:, index_cat_columns] - 1
# X_test_standardize[:, index_cat_columns] = X_test_standardize[:, index_cat_columns] - 1

# Question: do we need to standardize y_train or y_test?
# Fit the model
t_start = time.time() # Start the time
model.fit(X_train_standardize, y_train, iter_time=1000)
t_end = time.time() # End the timer

# Compute accuracy on testing set
y_pred_train = model.predict(X_train_standardize)
y_pred_test = model.predict(X_test_standardize)

accuracy_test = model.accu_eval(y_test, y_pred_test)
accuracy_train = model.accu_eval(y_train, y_pred_train)

print(f"Training accuracy: {100*accuracy_train:.2f}")
print(f"Testing accuracy: {100*accuracy_test:.2f}")
print(f"Executation time: {(t_end - t_start):.3f}s")

In [ ]:
# Implement k-fold cross validation

# Create an instance of the model object
initial_weights = np.zeros((X_hep_original.shape[1], 1))
model = LogisticClassifier(initial_weights)

# Question: Why doing this?  OK
X_hep_cat_change = X_hep_original.copy()
X_hep_cat_change[:, index_cat_columns] = X_hep_cat_change[:, index_cat_columns] - 1 

## Shuffling function will shuffle X&Y_hep_original, copy for kfold CV to use
X_hep_cat_kfold = X_hep_original.copy()
y_hep_cat_kfold = y_hep_original.copy()

# Perform cross-validation
cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list  = k_fold_Cross(model, X_hep_cat_kfold,
                                                     y_hep_cat_kfold, k=5, 
                                                     index_to_standardize=index_numerical_columns,
                                                     shuffle=True, printresult=True)

# print(f"It costs {(t_end-tStart)} seconds to finish k-fold cross validation")
plt.title('k-fold accuracy')
plt.xlabel('fold number'), plt.ylabel('accuracy')
plt.plot(cross_accuracy_list)
plt.show()

plt.title('k-fold cross entropy loss')
plt.xlabel('fold number'), plt.ylabel('CE')
plt.plot(CE_list)
plt.show()
"""
plt.title('k-fold iteration')
plt.xlabel('fold number'), plt.ylabel('iteration')
plt.plot(iteration_fold_list)
plt.show()
"""
print ("iteration time", iteration_fold_list)

In [ ]:
# Loop K-fold
k = 5
loop = 10
cross_accuracy = 0
iteration_fold = []
accuracy_fold = []
for i in range(loop):
  cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list = k_fold_Cross(model, X_hep_cat_kfold,
                                                     y_hep_cat_kfold, k=5, 
                                                     index_to_standardize=index_numerical_columns,
                                                     shuffle=True, printresult=False)
  cross_accuracy += sum(cross_accuracy_list)/k
  accuracy_fold.append(np.around((sum(cross_accuracy_list)/k)*100,2))
  iteration_fold.append(sum(iteration_fold_list)/k)
  

cross_accuracy = cross_accuracy/loop

print(f'Average cross accuracy is {100*cross_accuracy:.2f} %')
print (accuracy_fold)
print (iteration_fold)


# Basic fit and test of Bankruptcy data

In [ ]:
# split data into training and test sets
# !!!only need to to change data name here
"""
data_train, data_test = splitdata(bank_data, 0.8, True)

# pre-process trainding set (adding column 0 and standardization)
X_without_intercept = data_train.iloc[:,:-1].to_numpy() #all rows all columns except last column
x_train_pre = np.insert(X_without_intercept, 0, np.ones(X_without_intercept.shape[0]),
              axis=1)
x_train = x_train_pre
x_train[:, 1:] = standardization(x_train[:,1:],x_train_pre[:,1:])
y_train = data_train.iloc[:,-1].to_numpy().reshape(-1,1)

# predict with test set
X_without_intercept_test = data_test.iloc[:,:-1].to_numpy()
x_test = np.insert(X_without_intercept_test, 0, np.ones(X_without_intercept_test.shape[0]),
              axis=1)
x_test[:, 1:] = standardization(x_test[:,1:],x_train_pre[:,1:])
y_test = data_test.iloc[:,-1].to_numpy().reshape(-1,1)
w_init = np.zeros((x_train.shape[1], 1))
test = LogisticClassifier(w_init)
"""

X_bank_original_no_bias = bank_data.iloc[:, :-1].to_numpy()
X_bank_original = np.insert(
    X_bank_original_no_bias, 0, np.ones(X_bank_original_no_bias.shape[0]), axis=1
)
y_bank_original = bank_data.iloc[:, -1].to_numpy().reshape(-1, 1)

X_train, X_test = splitdata(X_bank_original, 0.8, False)
y_train, y_test = splitdata(y_bank_original, 0.8, False)

# Create an instance of the model object
initial_weights = np.zeros((X_train.shape[1], 1))
test = LogisticClassifier(initial_weights)

# Fit the model
t_start = time.time() # Start the time
Weight_store = test.fit(X_train, y_train, iter_time = 10000)
t_end = time.time() # End the timer

# Compute accuracy on testing set
y_pred_train = test.predict(X_train)
y_pred_test = test.predict(X_test)

accuracy_test = test.accu_eval(y_test, y_pred_test)
accuracy_train = test.accu_eval(y_train, y_pred_train)



print(f"Training accuracy: {100*accuracy_train:.2f}")
print(f"Testing accuracy: {100*accuracy_test:.2f}")
print(f"Executation time: {(t_end - t_start):.3f}s")


In [ ]:
# Implement k-fold cross validation
X_bank_original_no_bias = bank_data.iloc[:, :-1].to_numpy()
X_bank_original = np.insert(
    X_bank_original_no_bias, 0, np.ones(X_bank_original_no_bias.shape[0]), axis=1
)
y_bank_original = bank_data.iloc[:, -1].to_numpy().reshape(-1, 1)

# Create an instance of the model object
initial_weights = np.zeros((x_train.shape[1], 1))
test = LogisticClassifier(initial_weights)

## Shuffling function will shuffle X&Y_hep_original, copy for kfold CV to use
X_bank_kfold = X_bank_original.copy()
y_bank_kfold = y_bank_original.copy()

cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list  = k_fold_Cross(model, X_hep_cat_kfold,
                                                     y_hep_cat_kfold, k=5, 
                                                     index_to_standardize=index_numerical_columns,
                                                     shuffle=True, printresult=True)

# print(f"It costs {(t_end-tStart)} seconds to finish k-fold cross validation")
plt.title('k-fold accuracy')
plt.xlabel('fold number'), plt.ylabel('accuracy')
plt.plot(cross_accuracy_list)
plt.show()

plt.title('k-fold cross entropy loss')
plt.xlabel('fold number'), plt.ylabel('CE')
plt.plot(CE_list)
plt.show()
"""
plt.title('k-fold iteration')
plt.xlabel('fold number'), plt.ylabel('iteration')
plt.plot(iteration_fold_list)
plt.show()
"""
print ("iteration time", iteration_fold_list)

In [ ]:
# Loop K-fold
X_bank_kfold = X_bank_original.copy()
y_bank_kfold = y_bank_original.copy()
initial_weights = np.zeros((x_train.shape[1], 1))
model = LogisticClassifier(initial_weights)
k = 10
loop = 10
iteration_fold = []
accuracy_fold = []
cross_accuracy = 0
for i in range(loop):
  cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list= k_fold_Cross(model, X_bank_kfold,
                                                     y_bank_kfold, k=10, 
                                                     shuffle=True, printresult=False)
  cross_accuracy += sum(cross_accuracy_list)/k
  accuracy_fold.append(np.around((sum(cross_accuracy_list)/k)*100,2))
  iteration_fold.append(sum(iteration_fold_list)/k)
  

cross_accuracy = cross_accuracy/loop

print(f'Average cross accuracy is {100*cross_accuracy:.2f} %')
print (accuracy_fold)
print (iteration_fold)

# Compare effect of standardization (Template?)

Section hepatitis

In [ ]:
print("#####Section HEPATITIS")

# Indices of features
index_cat_columns = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18])
index_numerical_columns = np.array([0, 13, 14, 15, 16, 17])

# Create original set of features
X_hep_original_no_bias = hep_data.iloc[:, :-1].to_numpy()
X_hep_original = np.insert(
    X_hep_original_no_bias, 0, np.ones(X_hep_original_no_bias.shape[0]), axis=1
)
y_hep_original = hep_data.iloc[:, -1].to_numpy().reshape(-1, 1)

# Account for extra column for bias
index_numerical_columns = index_numerical_columns + 1
index_cat_columns = index_cat_columns + 1

#Test group 1
X_hep_cat_kfold = X_hep_original.copy()
y_hep_cat_kfold = y_hep_original.copy()

#Test group 2
X_hep_cat_kfold_standardized = X_hep_original.copy()
y_hep_cat_kfold_standardized = y_hep_original.copy()


# Loop K-fold group 1: NOT standardized
initial_weights = np.zeros((X_hep_original.shape[1], 1))
model = LogisticClassifier(initial_weights)

tstart1 = time.time()
k = 5
loop = 10
iteration_fold = []
accuracy_fold = []
cross_accuracy = 0
for i in range(loop):
  cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list  = k_fold_Cross(model, X_hep_cat_kfold,
                                                     y_hep_cat_kfold, k=5, 
                                                     index_to_standardize=index_numerical_columns,
                                                     standardize=False,
                                                     shuffle=True, printresult=False)
  cross_accuracy += sum(cross_accuracy_list)/k
  accuracy_fold.append(np.around((sum(cross_accuracy_list)/k)*100,2))
  iteration_fold.append(sum(iteration_fold_list)/k)

cross_accuracy = cross_accuracy/loop
tend1 = time.time()
print(f'Average cross accuracy is {100*cross_accuracy:.2f} %')
print (accuracy_fold)
print (iteration_fold)
print(f"It costs {(tend1-tstart1)} seconds")


# Loop K-fold group 2: standardized
initial_weights = np.zeros((X_hep_original.shape[1], 1))
model = LogisticClassifier(initial_weights)

tstart2=time.time()
k = 5
loop = 10
cross_accuracy = 0
iteration_fold = []
accuracy_fold = []
for i in range(loop):
  cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list = k_fold_Cross(model, X_hep_cat_kfold_standardized,
                                                     y_hep_cat_kfold_standardized, k=5,
                                                     index_to_standardize=index_numerical_columns,
                                                     standardize=True,
                                                     shuffle=True, printresult=False)
  cross_accuracy += sum(cross_accuracy_list)/k
  accuracy_fold.append(np.around((sum(cross_accuracy_list)/k)*100,2))
  iteration_fold.append(sum(iteration_fold_list)/k)

cross_accuracy = cross_accuracy/loop
tend2=time.time()
print(f'Average cross accuracy standardized is {100*cross_accuracy:.2f} %')
print (accuracy_fold)
print (iteration_fold)
print(f"It costs {(tend2-tstart2)} seconds")


Section Bankruptcy

In [ ]:
print("#####Section Bankruptcy")

# Implement k-fold cross validation
X_bank_original_no_bias = bank_data.iloc[:, :-1].to_numpy()
X_bank_original = np.insert(
    X_bank_original_no_bias, 0, np.ones(X_bank_original_no_bias.shape[0]), axis=1
)
y_bank_original = bank_data.iloc[:, -1].to_numpy().reshape(-1, 1)

# Test Group 1
X_bank_kfold = X_bank_original.copy()
y_bank_kfold = y_bank_original.copy()
# Test Group 2
X_bank_kfold_standardized = X_bank_original.copy()
y_bank_kfold_standardized = y_bank_original.copy()


# Loop K-fold group1: NOT standardized
initial_weights = np.zeros((x_train.shape[1], 1))
model = LogisticClassifier(initial_weights)

tstart1 = time.time()
k = 10
loop = 10
iteration_fold = []
accuracy_fold = []

cross_accuracy = 0
for i in range(loop):
  cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list= k_fold_Cross(model, X_bank_kfold,
                                                     y_bank_kfold, k=10, 
                                                     standardize=False,
                                                     shuffle=True, printresult=False)
  cross_accuracy += sum(cross_accuracy_list)/k
  accuracy_fold.append(np.around((sum(cross_accuracy_list)/k)*100,2))
  iteration_fold.append(sum(iteration_fold_list)/k)

cross_accuracy = cross_accuracy/loop
tend1 = time.time()
print(f'Average cross accuracy is {100*cross_accuracy:.2f} %')
print (accuracy_fold)
print (iteration_fold)
print(f"It costs {(tend1-tstart1)} seconds")


# Loop K-fold group2: standardized
initial_weights = np.zeros((x_train.shape[1], 1))
model = LogisticClassifier(initial_weights)

tstart2 = time.time()
k = 10
loop = 10
cross_accuracy = 0
iteration_fold = []
accuracy_fold = []
for i in range(loop):
  cross_accuracy_list, weight_fold_list, CE_list, iteration_fold_list = k_fold_Cross(model, X_bank_kfold_standardized,
                                                     y_bank_kfold_standardized, k=10,
                                                     standardize=False,                                                     
                                                     shuffle=True, printresult=False)
  cross_accuracy += sum(cross_accuracy_list)/k
  accuracy_fold.append(np.around((sum(cross_accuracy_list)/k)*100,2))
  iteration_fold.append(sum(iteration_fold_list)/k)

cross_accuracy = cross_accuracy/loop
tend2 = time.time()
print(f'Average cross accuracy is {100*cross_accuracy:.2f}%')
print (accuracy_fold)
print (iteration_fold)
print(f"It costs {(tend2-tstart2)} seconds")



# Adding features (Ignore for the moment!!)

In [ ]:
#add x^2 to all features in bankruptcy

rep = bank_data.shape[1]
for i in range(rep-1):
  bank_data.insert(bank_data.shape[1]-1, f'attribute{i+rep}', bank_data[f'attribute{i+1}']**2 )
print(bank_data.head())

# split data into training and test sets
data_train, data_test = splitdata(bank_data, 0.8)

# pre-process trainding set (adding column 0 and standardization)
X_without_intercept = data_train.iloc[:,:-1].to_numpy()
x_train = np.insert(X_without_intercept, 0, np.ones(X_without_intercept.shape[0]),
              axis=1)
x_train[:, 1:] = standardization(x_train[:,1:])
y_train = data_train.iloc[:,-1].to_numpy().reshape(-1,1)

# start clock
tStart = time.time()

# fit training set
w_init = np.zeros((x_train.shape[1], 1))
#w_init = np.random.rand(x_train.shape[1],1) * 100
test = LogisticClassifier(w_init)
test.fit(x_train, y_train, iter_time = 20000, epsilon= 1e-06)

# predict with test set
X_without_intercept_test = data_test.iloc[:,:-1].to_numpy()
x_test = np.insert(X_without_intercept_test, 0, np.ones(X_without_intercept_test.shape[0]),
              axis=1)
x_test[:, 1:] = standardization(x_test[:,1:])
y_test = data_test.iloc[:,-1].to_numpy().reshape(-1,1)
y_pred = test.predict(x_test)
#print(y_pred)

# find out accuracy of prediction
accuracy = test.accu_eval(y_test, y_pred)
print(f"The accuracy is {(accuracy)} ")

# end clock
tEnd = time.time()
print(f"It costs {(tEnd-tStart)} seconds")


# Removing independent features from the hepatitis data set

In [ ]:
# Create data

# Indices of features
index_cat_columns = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18])
index_numerical_columns = np.array([0, 13, 14, 15, 16, 17])

# Create original set of features
X_hep_original_no_bias = hep_data.iloc[:, :-1].to_numpy()
X_hep_original = np.insert(
    X_hep_original_no_bias, 0, np.ones(X_hep_original_no_bias.shape[0]), axis=1
)
y_hep_original = hep_data.iloc[:, -1].to_numpy().reshape(-1, 1)

# Shuffle data
full_array = np.concatenate((X_hep_original, y_hep_original), axis=1)
np.random.shuffle(full_array)
X_hep_original = full_array[:,:-1]
y_hep_original = full_array[:,[-1]]

# Account for extra column for bias
index_numerical_columns = index_numerical_columns + 1
index_cat_columns = index_cat_columns + 1

# indices of features to remove
features_to_remove = [2, 3, 4, 8, 9, 16]

initial_weights = np.zeros((X_hep_original.shape[1], 1))
model = LogisticClassifier(initial_weights)
cross_accuracy_list, weight_fold_list, CE_list = k_fold_Cross(model, 
                                                              X_hep_original, 
                                                              y_hep_original, 
                                                              k=5, 
                                                              standardize=False, 
                                                              shuffle=False, 
                                                              printresult=False)

print(np.mean(cross_accuracy_list))
  

for i in features_to_remove:
  print(f"Removing {hep_data.columns[i-1]}...")
  X_hep_remove = np.delete(X_hep_original, i, axis = 1)

  initial_weights = np.zeros((X_hep_remove.shape[1], 1))
  model = LogisticClassifier(initial_weights)

  cross_accuracy_list, weight_fold_list, CE_list = k_fold_Cross(model, 
                                                                X_hep_remove, 
                                                                y_hep_original, 
                                                                k=5, 
                                                                standardize=False, 
                                                                shuffle=False, 
                                                                printresult=False)
  
  print(np.mean(cross_accuracy_list))

# Remove all features
print(f"Removing all independent features...")
X_hep_remove = np.delete(X_hep_original, features_to_remove, axis=1)

initial_weights = np.zeros((X_hep_remove.shape[1], 1))
model = LogisticClassifier(initial_weights)

cross_accuracy_list, weight_fold_list, CE_list = k_fold_Cross(model, 
                                                              X_hep_remove, 
                                                              y_hep_original, 
                                                              k=5, 
                                                              standardize=False, 
                                                              shuffle=False, 
                                                              printresult=False)

print(np.mean(cross_accuracy_list))
